In [5]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import _init_paths

import os
import cv2

import argparse
from detectors.detector_factory import detector_factory

In [2]:
image_ext = ['jpg', 'jpeg', 'png', 'webp']
video_ext = ['mp4', 'mov', 'avi', 'mkv']
time_stats = ['tot', 'load', 'pre', 'net', 'dec', 'post', 'merge']

In [6]:
parser = argparse.ArgumentParser()
# basic experiment setting
parser.add_argument('task', default='ctdet',
                             help='ctdet | ddd | multi_pose | exdet')
'''parser.add_argument('--dataset', default='coco',
                             help='coco | kitti | coco_hp | pascal')
parser.add_argument('--exp_id', default='default')
parser.add_argument('--test', action='store_true')
parser.add_argument('--debug', type=int, default=0,
                             help='level of visualization.'
                                  '1: only show the final detection results'
                                  '2: show the network output features'
                                  '3: use matplot to display' # useful when lunching training with ipython notebook
                                  '4: save all visualizations to disk')
parser.add_argument('--demo', default='', 
                             help='path to image/ image folders/ video. '
                                  'or "webcam"')
parser.add_argument('--load_model', default='',
                             help='path to pretrained model')
parser.add_argument('--resume', action='store_true',
                             help='resume an experiment. '
                                  'Reloaded the optimizer parameter and '
                                  'set load_model to model_last.pth '
                                  'in the exp dir if load_model is empty.') 

# system
parser.add_argument('--gpus', default='0', 
                             help='-1 for CPU, use comma for multiple gpus')
parser.add_argument('--num_workers', type=int, default=4,
                             help='dataloader threads. 0 for single-thread.')
parser.add_argument('--not_cuda_benchmark', action='store_true',
                             help='disable when the input size is not fixed.')
parser.add_argument('--seed', type=int, default=317, 
                             help='random seed') # from CornerNet

# log
parser.add_argument('--print_iter', type=int, default=0, 
                             help='disable progress bar and print to screen.')
parser.add_argument('--hide_data_time', action='store_true',
                             help='not display time during training.')
parser.add_argument('--save_all', action='store_true',
                             help='save model to disk every 5 epochs.')
parser.add_argument('--metric', default='loss', 
                             help='main metric to save best model')
parser.add_argument('--vis_thresh', type=float, default=0.3,
                             help='visualization threshold.')
parser.add_argument('--debugger_theme', default='white', 
                             choices=['white', 'black'])
    
# model
parser.add_argument('--arch', default='dla_34', 
                             help='model architecture. Currently tested'
                                  'res_18 | res_101 | resdcn_18 | resdcn_101 |'
                                  'dlav0_34 | dla_34 | hourglass')
parser.add_argument('--head_conv', type=int, default=-1,
                             help='conv layer channels for output head'
                                  '0 for no conv layer'
                                  '-1 for default setting: '
                                  '64 for resnets and 256 for dla.')
parser.add_argument('--down_ratio', type=int, default=4,
                             help='output stride. Currently only supports 4.')

# input
parser.add_argument('--input_res', type=int, default=-1, 
                             help='input height and width. -1 for default from '
                             'dataset. Will be overriden by input_h | input_w')
parser.add_argument('--input_h', type=int, default=-1, 
                             help='input height. -1 for default from dataset.')
parser.add_argument('--input_w', type=int, default=-1, 
                             help='input width. -1 for default from dataset.')
    
# train
parser.add_argument('--lr', type=float, default=1.25e-4, 
                             help='learning rate for batch size 32.')
parser.add_argument('--lr_step', type=str, default='90,120',
                             help='drop learning rate by 10.')
parser.add_argument('--num_epochs', type=int, default=140,
                             help='total training epochs.')
parser.add_argument('--batch_size', type=int, default=32,
                             help='batch size')
parser.add_argument('--master_batch_size', type=int, default=-1,
                             help='batch size on the master gpu.')
parser.add_argument('--num_iters', type=int, default=-1,
                             help='default: #samples / batch_size.')
parser.add_argument('--val_intervals', type=int, default=5,
                             help='number of epochs to run validation.')
parser.add_argument('--trainval', action='store_true',
                             help='include validation in training and '
                                  'test on test set')

# test
parser.add_argument('--flip_test', action='store_true',
                             help='flip data augmentation.')
parser.add_argument('--test_scales', type=str, default='1',
                             help='multi scale test augmentation.')
parser.add_argument('--nms', action='store_true',
                             help='run nms in testing.')
parser.add_argument('--K', type=int, default=100,
                             help='max number of output objects.') 
parser.add_argument('--not_prefetch_test', action='store_true',
                             help='not use parallal data pre-processing.')
parser.add_argument('--fix_res', action='store_true',
                             help='fix testing resolution or keep '
                                  'the original resolution')
parser.add_argument('--keep_res', action='store_true',
                             help='keep the original resolution'
                                  ' during validation.')

# dataset
parser.add_argument('--not_rand_crop', action='store_true',
                             help='not use the random crop data augmentation'
                                  'from CornerNet.')
parser.add_argument('--shift', type=float, default=0.1,
                             help='when not using random crop'
                                  'apply shift augmentation.')
parser.add_argument('--scale', type=float, default=0.4,
                             help='when not using random crop'
                                  'apply scale augmentation.')
parser.add_argument('--rotate', type=float, default=0,
                             help='when not using random crop'
                                  'apply rotation augmentation.')
parser.add_argument('--flip', type = float, default=0.5,
                             help='probability of applying flip augmentation.')
parser.add_argument('--no_color_aug', action='store_true',
                             help='not use the color augmenation '
                                  'from CornerNet')
# multi_pose
parser.add_argument('--aug_rot', type=float, default=0, 
                             help='probability of applying '
                                  'rotation augmentation.')
# ddd
parser.add_argument('--aug_ddd', type=float, default=0.5,
                             help='probability of applying crop augmentation.')
parser.add_argument('--rect_mask', action='store_true',
                             help='for ignored object, apply mask on the '
                                  'rectangular region or just center point.')
parser.add_argument('--kitti_split', default='3dop',
                             help='different validation split for kitti: '
                                  '3dop | subcnn')

# loss
parser.add_argument('--mse_loss', action='store_true',
                             help='use mse loss or focal loss to train '
                                  'keypoint heatmaps.')
# ctdet
parser.add_argument('--reg_loss', default='l1',
                             help='regression loss: sl1 | l1 | l2')
parser.add_argument('--hm_weight', type=float, default=1,
                             help='loss weight for keypoint heatmaps.')
parser.add_argument('--off_weight', type=float, default=1,
                             help='loss weight for keypoint local offsets.')
parser.add_argument('--wh_weight', type=float, default=0.1,
                             help='loss weight for bounding box size.')
# multi_pose
parser.add_argument('--hp_weight', type=float, default=1,
                             help='loss weight for human pose offset.')
parser.add_argument('--hm_hp_weight', type=float, default=1,
                             help='loss weight for human keypoint heatmap.')
# ddd
parser.add_argument('--dep_weight', type=float, default=1,
                             help='loss weight for depth.')
parser.add_argument('--dim_weight', type=float, default=1,
                             help='loss weight for 3d bounding box size.')
parser.add_argument('--rot_weight', type=float, default=1,
                             help='loss weight for orientation.')
parser.add_argument('--peak_thresh', type=float, default=0.2)
    
# task
# ctdet
parser.add_argument('--norm_wh', action='store_true',
                             help='L1(\hat(y) / y, 1) or L1(\hat(y), y)')
parser.add_argument('--dense_wh', action='store_true',
                             help='apply weighted regression near center or '
                                  'just apply regression on center point.')
parser.add_argument('--cat_spec_wh', action='store_true',
                             help='category specific bounding box size.')
parser.add_argument('--not_reg_offset', action='store_true',
                             help='not regress local offset.')
# exdet
parser.add_argument('--agnostic_ex', action='store_true',
                             help='use category agnostic extreme points.')
parser.add_argument('--scores_thresh', type=float, default=0.1,
                             help='threshold for extreme point heatmap.')
parser.add_argument('--center_thresh', type=float, default=0.1,
                             help='threshold for centermap.')
parser.add_argument('--aggr_weight', type=float, default=0.0,
                             help='edge aggregation weight.')
# multi_pose
parser.add_argument('--dense_hp', action='store_true',
                             help='apply weighted pose regression near center '
                                  'or just apply regression on center point.')
parser.add_argument('--not_hm_hp', action='store_true',
                             help='not estimate human joint heatmap, '
                                  'directly use the joint offset from center.')
parser.add_argument('--not_reg_hp_offset', action='store_true',
                             help='not regress local offset for '
                                  'human joint heatmaps.')
parser.add_argument('--not_reg_bbox', action='store_true',
                             help='not regression bounding box size.')
    
# ground truth validation
parser.add_argument('--eval_oracle_hm', action='store_true', 
                             help='use ground center heatmap.')
parser.add_argument('--eval_oracle_wh', action='store_true', 
                             help='use ground truth bounding box size.')
parser.add_argument('--eval_oracle_offset', action='store_true', 
                             help='use ground truth local heatmap offset.')
parser.add_argument('--eval_oracle_kps', action='store_true', 
                             help='use ground truth human pose offset.')
parser.add_argument('--eval_oracle_hmhp', action='store_true', 
                             help='use ground truth human joint heatmaps.')
parser.add_argument('--eval_oracle_hp_offset', action='store_true', 
                             help='use ground truth human joint local offset.')
parser.add_argument('--eval_oracle_dep', action='store_true', 
                             help='use ground truth depth.') '''

'parser.add_argument(\'--dataset\', default=\'coco\',\n                             help=\'coco | kitti | coco_hp | pascal\')\nparser.add_argument(\'--exp_id\', default=\'default\')\nparser.add_argument(\'--test\', action=\'store_true\')\nparser.add_argument(\'--debug\', type=int, default=0,\n                             help=\'level of visualization.\'\n                                  \'1: only show the final detection results\'\n                                  \'2: show the network output features\'\n                                  \'3: use matplot to display\' # useful when lunching training with ipython notebook\n                                  \'4: save all visualizations to disk\')\nparser.add_argument(\'--demo\', default=\'\', \n                             help=\'path to image/ image folders/ video. \'\n                                  \'or "webcam"\')\nparser.add_argument(\'--load_model\', default=\'\',\n                             help=\'path to pretrained model\')\

In [7]:
args = parser.parse_args(args=[])
print(args)

usage: ipykernel_launcher.py [-h] task
ipykernel_launcher.py: error: the following arguments are required: task


SystemExit: 2

In [ ]:
yy_opt = opt.parse()

In [ ]:
%tb

In [ ]:
print(opt)

In [ ]:
opt.eval_oracle_hmhp


In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = opt.gpus_str
opt.debug = max(opt.debug, 1)
Detector = detector_factory[opt.task]
detector = Detector(opt)

In [ ]:
if opt.demo == 'webcam' or \
    opt.demo[opt.demo.rfind('.') + 1:].lower() in video_ext:
    cam = cv2.VideoCapture(0 if opt.demo == 'webcam' else opt.demo)
    detector.pause = False
    while True:
        _, img = cam.read()
        cv2.imshow('input', img)
        ret = detector.run(img)
        time_str = ''
        for stat in time_stats:
          time_str = time_str + '{} {:.3f}s |'.format(stat, ret[stat])
        print(time_str)
        if cv2.waitKey(1) == 27:
            return  # esc to quit
else:
    if os.path.isdir(opt.demo):
      image_names = []
      ls = os.listdir(opt.demo)
      for file_name in sorted(ls):
          ext = file_name[file_name.rfind('.') + 1:].lower()
          if ext in image_ext:
              image_names.append(os.path.join(opt.demo, file_name))
    else:
      image_names = [opt.demo]
    
    for (image_name) in image_names:
      ret = detector.run(image_name)
      time_str = ''
      for stat in time_stats:
        time_str = time_str + '{} {:.3f}s |'.format(stat, ret[stat])
      print(time_str)

